### Non-masked

In [1]:
import cobra
from cobra.io import load_json_model
import numpy as np
import pandas as pd
import ray
cobra_model = load_json_model("./COBRA_models/GEM_Recon2_thermocurated_redHUMAN.json")
import itertools

def grow_dict(previo, nuevo):  
    """Esto es un .update(overwritte=False)?"""

    not_new_keys = set(previo.keys()).intersection(set(nuevo.keys()))
    new_keys     = set(nuevo.keys()).difference(set(previo.keys()))
    for new in new_keys:
        previo[new] =np.unique(nuevo[new]).tolist() 
    for common in not_new_keys:
        previo[common].extend(np.unique(nuevo[common]).tolist() )
    return previo
# Sacamos los genes de Recon hacia un diccionario
# TODO: convertir est en una horrible dict_comprehension { curse : cursed for c in curseds }
dict_rxn_to_gene = dict()
genes            = [g.id for g in cobra_model.genes]

for id in genes:
    # fmt: off
    rxns             = [r.id for r in cobra_model.genes.get_by_id(id)._reaction] # reacciones asociadas al gen de la iter
    updated_dict  : dict[str, list[str]] = dict(zip(rxns,itertools.repeat([id]))) # zipping con la { reaccion : [id_gen] }
    dict_rxn_to_gene = grow_dict(dict_rxn_to_gene, updated_dict) # merge recursivo

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [15]:
Non_masked_edges_node_names_explainer_subgraphs = pd.read_parquet(
    "./results/dataframes/Non_masked_edges_node_names_explainer_subgraphs.parquet.gzip").reset_index(drop=True)

ray.shutdown()
ray.init()
@ray.remote


def get_genes(node_1, node_2):
    
    if node_2 in dict_rxn_to_gene.keys():
        
        return dict_rxn_to_gene[node_2]
        
    elif node_1 in dict_rxn_to_gene.keys():
        
        return dict_rxn_to_gene[node_1]
        
    else:
        return ['']
    
g = [] 
for node_1, node_2 in zip(Non_masked_edges_node_names_explainer_subgraphs.node1.tolist(), Non_masked_edges_node_names_explainer_subgraphs.node2.tolist()):   
    g.append(get_genes.remote(node_1, node_2 ))

 
genes = pd.Series(ray.get(g) , name='genes')



assert genes.shape[0] == Non_masked_edges_node_names_explainer_subgraphs.shape[0]



Non_masked_results_for_Heatmaps = pd.concat([genes, Non_masked_edges_node_names_explainer_subgraphs], axis=1)
Non_masked_results_for_Heatmaps


2022-11-08 18:23:44,770	INFO worker.py:1518 -- Started a local Ray instance.


,genes,node1,node2,0,1,2,3,4,5,6,...,4911,4912,4913,4914,4915,4916,4917,4918,4919,4920
0,[],GLYtp,gly_x,0.809316,0.017071,0.944399,0.014294,0.013697,0.014400,0.016846,...,0.014113,0.023876,0.939117,0.014372,0.924866,0.827539,0.014083,0.913459,0.013974,0.013647
1,[9057.1],r1775,phe_L_e,0.009859,0.038082,0.008392,0.069493,0.104697,0.054944,0.045725,...,0.051504,0.041776,0.008341,0.054734,0.010928,0.008352,0.054910,0.008472,0.054533,0.100641
2,[9057.1],r1764,phe_L_e,0.009074,0.037087,0.008323,0.070738,0.102954,0.056667,0.044050,...,0.051922,0.035115,0.008064,0.054673,0.008636,0.008498,0.057313,0.008594,0.054202,0.102157
3,"[56301.1, 6520.1]",SERDGLYexR,ser_D_e,0.012321,0.018849,0.013958,0.014370,0.013431,0.015026,0.020074,...,0.014359,0.029213,0.013197,0.014674,0.013804,0.013585,0.014264,0.012541,0.014184,0.013492
4,"[56301.1, 6520.1]",SERDGLYexR,ser_D_c,0.013410,0.013671,0.013923,0.014184,0.013575,0.014927,0.019511,...,0.013654,0.027464,0.013792,0.014596,0.013832,0.013788,0.014235,0.013711,0.014025,0.013558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,[25828.1],glu_L_c,r1294,0.011244,0.079504,0.013631,0.094449,0.016433,0.244240,0.469874,...,0.036766,0.505658,0.013501,0.821198,0.014351,0.013732,0.073615,0.012710,0.174806,0.014933
381,[],glu_L_c,r1350,0.010739,0.065239,0.012363,0.077868,0.015230,0.149757,0.356885,...,0.032906,0.319373,0.012532,0.588149,0.013105,0.013051,0.058200,0.011912,0.108694,0.014540
382,[],glu_L_c,r1339,0.010424,0.081280,0.012305,0.106227,0.015265,0.249086,0.564752,...,0.037034,0.602232,0.012227,0.835052,0.012747,0.012640,0.080240,0.011520,0.180944,0.014614
383,[4706.1],glu_L_c,r1096,0.010930,0.094387,0.013328,0.122039,0.018642,0.302735,0.615896,...,0.040938,0.641238,0.013157,0.864087,0.014171,0.013575,0.120887,0.012394,0.237277,0.015303


In [16]:
Non_masked_results_for_Heatmaps.to_csv(
    "./results/dataframes/Non_masked_results_for_Heatmaps.csv")

# Masked

In [17]:
Masked_edges_node_names_explainer_subgraphs = pd.read_parquet(
    "./results/dataframes/Masked_edges_node_names_explainer_subgraphs.parquet.gzip").reset_index(drop=True)

ray.shutdown()
ray.init()
@ray.remote


def get_genes(node_1, node_2):
    
    if node_2 in dict_rxn_to_gene.keys():
        
        return dict_rxn_to_gene[node_2]
        
    elif node_1 in dict_rxn_to_gene.keys():
        
        return dict_rxn_to_gene[node_1]
        
    else:
        return ['']
    
g = [] 
for node_1, node_2 in zip(Masked_edges_node_names_explainer_subgraphs.node1.tolist(), Masked_edges_node_names_explainer_subgraphs.node2.tolist()):   
    g.append(get_genes.remote(node_1, node_2 ))

 
genes = pd.Series(ray.get(g) , name='genes')



assert genes.shape[0] == Masked_edges_node_names_explainer_subgraphs.shape[0]



Masked_results_for_Heatmaps = pd.concat([genes, Masked_edges_node_names_explainer_subgraphs], axis=1)
Masked_results_for_Heatmaps

2022-11-08 20:00:35,749	INFO worker.py:1518 -- Started a local Ray instance.


,genes,node1,node2,0,1,2,3,4,5,6,...,4911,4912,4913,4914,4915,4916,4917,4918,4919,4920
0,[2936.1],GTHO,gthrd_c,0.263610,0.013635,0.255285,0.013854,0.292490,0.013644,0.291774,...,0.013862,0.280309,0.264044,0.284307,0.156630,0.294427,0.321659,0.238522,0.013877,0.013732
1,"[38.1, 3032.1, 3030.1]",ACACT1rm,coa_m,0.500152,0.013634,0.511573,0.013680,0.614619,0.013671,0.587768,...,0.013698,0.618933,0.623276,0.620856,0.603076,0.595013,0.602163,0.639294,0.013622,0.013867
2,[6566.1],r2107,akg_c,0.022213,0.013784,0.174154,0.013734,0.358388,0.013767,0.400256,...,0.013817,0.105676,0.230749,0.342714,0.112324,0.490835,0.379571,0.433610,0.013769,0.013707
3,[290.1],cgly_c,AMPTASECG,0.013658,0.014185,0.565478,0.014103,0.488262,0.014245,0.590854,...,0.014256,0.558333,0.014041,0.473315,0.580545,0.470971,0.578085,0.562015,0.014133,0.014127
4,[2243.1],pro_L_c,r1101,0.044910,0.140470,0.269273,0.093496,0.032427,0.888334,0.132530,...,0.096484,0.113944,0.033161,0.114167,0.074234,0.061514,0.067508,0.074055,0.034199,0.027921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,[],glu_L_c,r1339,0.342187,0.014052,0.110080,0.920189,0.145110,0.066037,0.254301,...,0.084680,0.168148,0.250982,0.112418,0.214045,0.251329,0.195160,0.238165,0.022755,0.050282
381,[2356.1],glu_L_c,FPGS7,0.203736,0.013731,0.393837,0.013662,0.446415,0.013726,0.265397,...,0.013647,0.360433,0.286605,0.463111,0.319126,0.340212,0.348344,0.448906,0.013639,0.013859
382,[],glu_L_c,RE2034C,0.494993,0.013670,0.160197,0.014753,0.373386,0.014423,0.471495,...,0.014667,0.397345,0.457903,0.391696,0.438482,0.431346,0.376282,0.367636,0.014157,0.014622
383,[4706.1],glu_L_c,r1096,0.346614,0.023622,0.119780,0.973370,0.154909,0.903673,0.274275,...,0.958604,0.199790,0.266435,0.118575,0.253278,0.307636,0.210369,0.258740,0.043224,0.917637


In [18]:
Masked_results_for_Heatmaps.to_csv(
    "./results/dataframes/Masked_results_for_Heatmaps.csv")